### Get Access Token

In [ ]:
import robobrowser
import re

In [ ]:
MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; U; en-gb; KFTHWI Build/JDQ39) AppleWebKit/535.19 (KHTML, like Gecko) Silk/3.16 Safari/535.19"
FB_AUTH = "https://www.facebook.com/v2.6/dialog/oauth?redirect_uri=fb464891386855067%3A%2F%2Fauthorize%2F&display=touch&state=%7B%22challenge%22%3A%22IUUkEUqIGud332lfu%252BMJhxL4Wlc%253D%22%2C%220_auth_logger_id%22%3A%2230F06532-A1B9-4B10-BB28-B29956C71AB1%22%2C%22com.facebook.sdk_client_state%22%3Atrue%2C%223_method%22%3A%22sfvc_auth%22%7D&scope=user_birthday%2Cuser_photos%2Cuser_education_history%2Cemail%2Cuser_relationship_details%2Cuser_friends%2Cuser_work_history%2Cuser_likes&response_type=token%2Csigned_request&default_audience=friends&return_scopes=true&auth_type=rerequest&client_id=464891386855067&ret=login&sdk=ios&logger_id=30F06532-A1B9-4B10-BB28-B29956C71AB1&ext=1470840777&hash=AeZqkIcf-NEW6vBd"

s = robobrowser.RoboBrowser(user_agent=MOBILE_USER_AGENT, parser="lxml")
s.open(FB_AUTH)

# submit login form
f = s.get_form()
f["pass"] = ?
f["email"] = ?
s.submit_form(f)

# click the 'ok' button on the dialog
f = s.get_form()
s.submit_form(f, submit=f.submit_fields['__CONFIRM__'])

# get access token from the html response
access_token = re.search(r"access_token=([\w\d]+)", s.response.content.decode()).groups()[0]

### Imports

In [ ]:
import pynder
import socket
socket.setdefaulttimeout(60)

from IPython.display import Image, HTML, display, clear_output
from urllib.request import urlretrieve

import os
import time
import warnings

import numpy as np
import pandas as pd

import tensorflow as tf

from facenet.src import facenet
from facenet.src.align import detect_face

import imageio
warnings.filterwarnings("ignore", module="imageio")
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

### Make Session

In [ ]:
session = pynder.Session(access_token)
session.banned

### Get Nearby Users

In [ ]:
users = session.nearby_users()

### Scrape Photos and Label Preferences

In [ ]:
if not os.path.exists('images'):
    os.makedirs('images')
if not os.path.exists('images/like'):
    os.makedirs('images/like')
if not os.path.exists('images/dislike'):
    os.makedirs('images/dislike')

In [ ]:
def move():
    os.chdir("images")
    for photo in os.listdir():
        if photo.endswith('.jpg'):
            if res == '1':
                os.rename(photo,'like/'+photo)
            elif res == '2':
                os.rename(photo,'superlike/'+photo)
            elif res == '0':
                os.rename(photo,'dislike/'+photo)
    
    os.chdir("..")

for g in range(1):
    clear_output(wait=False)
    girl = next(users)
    print(girl.name)
    photos = list(girl.get_photos(width=640))[:5]
    for photo in photos:
        urlretrieve(photo, 'images/'+str(photo).split('/')[-1])
    photos = list(girl.get_photos(width=172))[:5]
    imghtml=''.join( ["<img style='float: left; margin: 0px;' src='%s'/>" % str(photo) 
                         for photo in photos])
    display(HTML(imghtml))
    res = input()
    if res == '1':
        girl.like()
        move()
    elif res == '2':
        girl.superlike()
        move()
    elif res == '0':
        girl.dislike()
        move()

### Extract Faces and Align Images

In [ ]:
if not os.path.exists('images_aligned'):
    os.makedirs('images_aligned')

In [ ]:
dataset = facenet.get_dataset('images')

# networks and loading parameters
with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = detect_face.create_mtcnn(sess, None)

In [ ]:
for cls in dataset:
    if not os.path.exists(os.path.join('images_aligned', cls.name)):
        os.makedirs(os.path.join('images_aligned', cls.name))      
    for image_path in cls.image_paths:
        filename = os.path.join('images_aligned', cls.name, os.path.splitext(os.path.split(image_path)[1])[0]+'.png')
        img = imageio.imread(image_path)[:,:,0:3]
        bb, _ = detect_face.detect_face(img, 20, pnet, rnet, onet, [0.6,0.7,0.7 ], 0.709)
        if bb.shape[0]>0:
            det = bb[:,0:4]
            imgsize = np.asarray(img.shape)[0:2]
            if bb.shape[0]>1:
                bbsize = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                imgcenter = imgsize/2
                offsets = np.vstack([ (det[:,0]+det[:,2])/2-imgcenter[1], (det[:,1]+det[:,3])/2-imgcenter[0]])
                det = det[np.argmax(bbsize-np.sum(np.power(offsets,2.0),0)*2.0),:]
            det = np.squeeze(det)
            bb = np.zeros(4, dtype=np.int32)
            bb[0] = np.maximum(det[0]-44/2, 0)
            bb[1] = np.maximum(det[1]-44/2, 0)
            bb[2] = np.minimum(det[2]+44/2, imgsize[1])
            bb[3] = np.minimum(det[3]+44/2, imgsize[0])
            cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
            scaled = resize(cropped, (182, 182), mode='constant')
            imageio.imwrite(filename, scaled)

### Export Embeddings

In [ ]:
train = facenet.get_dataset('images_aligned')
images, labels = facenet.get_image_paths_and_labels(train)
 
with tf.Graph().as_default():
    with tf.Session() as sess:
        facenet.load_model('20180402-114759')

        imgplaceholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        images = facenet.load_data(images, False, False, 160)
        feed_dict = {imgplaceholder: images, phase_train:False}
        embeddings = sess.run(embeddings, feed_dict=feed_dict)

        np.save('embeddings.npy', embeddings)
        np.save('labels.npy', labels)

### Load Saved Embeddings

In [ ]:
embeddings=np.load('embeddings.npy')
labels=np.load('labels.npy')

tinderfacenet = pd.DataFrame(embeddings)
tinderfacenet['labels']=labels

### Train/Test Split

In [ ]:
trainX, testX, trainY, testY = train_test_split(tinderfacenet.drop(columns=['labels']), tinderfacenet['labels'], 
                                                test_size=0.3)
trainX.shape, testX.shape, trainY.shape, testY.shape

### Model Building

In [ ]:
trainY.value_counts()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# model = XGBClassifier()
# model = GradientBoostingClassifier(loss='exponential')
model = GradientBoostingClassifier(loss='deviance')

model.fit(trainX, trainY)
print(model.score(testX, testY))

preds = model.predict(testX)
cm = confusion_matrix(testY, preds)
tn, fp, fn, tp = cm.ravel()
recall = float(tp) / np.sum(cm,axis=1)[1]
specificity = float(tn) /np.sum(cm,axis=1)[0]

print('like accuracy: ', recall)
print('dislike accuracy: ', specificity)

# joblib.dump(model, 'GBdev_model.pkl')
# joblib.dump(model, 'GBexp_model.pkl')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


def create_baseline():
    model = Sequential()
    model.add(Dense(512, input_dim=512, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
results = cross_val_score(estimator, trainX, trainY, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Inteligent Swipe

In [ ]:
session.profile.distance_filter 

In [ ]:
# session.update_location(LAT, LON)
session.profile.pos

In [ ]:
model = joblib.load('GBdev_model.pkl')
users = session.nearby_users()

In [ ]:
if not os.path.exists('temp'):
    os.makedirs('temp')
if not os.path.exists('temp_aligned'):
    os.makedirs('temp_aligned')

def clean(folder):
    for file in os.listdir(folder):
        os.unlink(os.path.join(folder, file))
        
def show():
    photos = list(girl.get_photos(width=172))[:5]
    imghtml=''.join( ["<img style='float: left; margin: 0px;' src='%s'/>" % str(photo) 
                         for photo in photos])
    display(HTML(imghtml))

In [ ]:
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with tf.Session() as sess:
        pnet, rnet, onet = detect_face.create_mtcnn(sess, None)
        facenet.load_model('20180402-114759')
        
        for girl in users:
            clear_output(wait=True)
            photos = list(girl.get_photos(width=640))[:5]
            for photo in photos:
                urlretrieve(photo, 'temp/'+str(photo).split('/')[-1])
            
            for photo in os.listdir('temp'):
                img = imageio.imread('temp/'+photo)[:,:,0:3]
                bb, _ = detect_face.detect_face(img, 20, pnet, rnet, onet, [0.6,0.7,0.7 ], 0.709)
                if bb.shape[0]>0:
                    det = bb[:,0:4]
                    imgsize = np.asarray(img.shape)[0:2]
                    if bb.shape[0]>1:
                        bbsize = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                        imgcenter = imgsize/2
                        offsets = np.vstack([ (det[:,0]+det[:,2])/2-imgcenter[1], (det[:,1]+det[:,3])/2-imgcenter[0]])
                        det = det[np.argmax(bbsize-np.sum(np.power(offsets,2.0),0)*2.0),:]
                    det = np.squeeze(det)
                    bb = np.zeros(4, dtype=np.int32)
                    bb[0] = np.maximum(det[0]-44/2, 0)
                    bb[1] = np.maximum(det[1]-44/2, 0)
                    bb[2] = np.minimum(det[2]+44/2, imgsize[1])
                    bb[3] = np.minimum(det[3]+44/2, imgsize[0])
                    cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                    scaled = resize(cropped, (182, 182), mode='constant')
                    imageio.imwrite('temp_aligned/'+photo, scaled)

            imgplaceholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            if len(os.listdir('temp_aligned'))==0:
                girl.dislike()
                continue
            os.chdir('temp_aligned')
            images = facenet.load_data(os.listdir(), False, False, 160)
            os.chdir('..')
            feed_dict = {imgplaceholder: images, phase_train:False}
            embeddings = pd.DataFrame(sess.run(embeddings, feed_dict=feed_dict))
                    
            preds = model.predict(embeddings)
            label, counts = np.unique(preds, return_counts=True)
            
            dec = {}
            dec[0]=0
            dec[1]=0
            dec.update(dict(zip(label, counts)))
            
            if dec[1]>dec[0]:
                girl.like()
                show()
                print("liked")
                time.sleep(5)
            else:
                girl.dislike()
                show()
                print("disliked")
                time.sleep(5)
            
            clean('temp')
            clean('temp_aligned')